# Supplement 1 - Abbreviation table

In [14]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../../functionScripts/')
sys.path.append('../../Atlas/')

import helperFunctions as hf

# Variables
dataFeature = 'abbreviation'

# Load files
lightsheet_data = pd.read_pickle('..//..//Temp//lightsheet_all.pkl') 
aba_ccf = pd.read_excel('../../Atlas/ABA_CCF.xlsx')

# extract abbreviation and structure ID from aba_cff
aba_ccf = aba_ccf[['structure ID', 'abbreviation']]
regionArea = hf.create_region_to_area_dict(lightsheet_data, dataFeature).reset_index(drop=True)

# merge aba_ccf into regionArea using abbreviation, Drop regions with NaN for Brain_Area
regionArea_aba = regionArea.merge(aba_ccf, on='abbreviation', how='outer').dropna(subset=['Brain_Area'])

lightsheet_data_orig = lightsheet_data.copy()
lightsheet_data_orig = lightsheet_data.groupby(['dataset', 'sex'])['count_norm'].sum()

# Generate tables for merging
lsData = lightsheet_data[['abbreviation', 'Region_Name', 'volume_(mm^3)']]
regionArea2 = regionArea_aba.merge(lsData, on=dataFeature, how='outer').drop_duplicates(subset=['abbreviation'])

# Rename the structure ID column to region_ID
regionArea2.rename(columns={'structure ID': 'region ID'}, inplace=True)
regionArea2.rename(columns={'Region_Name': 'region name'}, inplace=True)
regionArea2.rename(columns={'Brain_Area': 'brain area'}, inplace=True)
regionArea2.rename(columns={'volume_(mm^3)': 'volume (mm^3)'}, inplace=True)

# Resort the columns, drop the extra
regionAreaCSV = regionArea2[['region name', 'volume (mm^3)', 'region ID', 'abbreviation', 'brain area']].reset_index(drop=True)

# Round the volume column to 4 decimal places
regionAreaCSV['volume (mm^3)'] = regionAreaCSV['volume (mm^3)'].round(4)

# Optional editting
# regionAreaCSV.columns = regionAreaCSV.columns.str.replace('_', ' ')
# regionAreaCSV.columns = [x.title() for x in regionAreaCSV.columns]

# Save to a csv file
regionAreaCSV.to_csv('suppRegionTable.csv', index=False)

c:\Users\aboha\anaconda3\envs\cFosProject\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


# Supplementary Table 2 

In [15]:
# Create a table with the columns abbreviation, Region_Name, and the mean across each of the drugs in the dataset

# Get the columns that are not the abbreviation or the region name
cols = lightsheet_data.columns[~lightsheet_data.columns.isin(['abbreviation', 'Region_Name'])]

# For each of the datasets, replace '6FDET' with '6-F-DET'
lightsheet_data['dataset'] = lightsheet_data['dataset'].str.replace('6FDET', '6-F-DET')

# Create a pivot table 
ls_pivot = lightsheet_data.pivot(index=dataFeature, columns='dataset', values='count')

# Add the word ' count' to each of the columns
ls_pivot.columns = [x + ' count' for x in ls_pivot.columns]

# Add a column for the Region_Name according to abbreviation, make it the second column
columnNames = ls_pivot.columns
ls_pivot['region name'] = ls_pivot.index.map(regionArea2.set_index(dataFeature)['region name'])
ls_pivot = ls_pivot[['region name'] + list(columnNames)]

# Resort according to Areas
resorting_idx = regionArea[dataFeature]
ls_pivot = ls_pivot.loc[resorting_idx]

# Save to a csv file
ls_pivot.to_csv('suppTable_all.csv')

In [16]:
# Take a mean across each of the drug values for each region
lightsheet_data_mean = lightsheet_data.groupby(['abbreviation', 'drug'])['count'].mean().reset_index()

# Pivot the table
lightsheet_data_mean_pivot = lightsheet_data_mean.pivot(index='abbreviation', columns='drug', values='count')

lightsheet_data_mean_pivot.columns = [x + ' mean count' for x in lightsheet_data_mean_pivot.columns]

# Add a column for the Region_Name according to abbreviation, make it the second column
columnNames = lightsheet_data_mean_pivot.columns
lightsheet_data_mean_pivot['region name'] = lightsheet_data_mean_pivot.index.map(regionArea2.set_index(dataFeature)['region name'])
lightsheet_data_mean_pivot = lightsheet_data_mean_pivot[['region name'] + list(columnNames)]

# Resort according to Areas
resorting_idx = regionArea[dataFeature]
lightsheet_data_mean_pivot = lightsheet_data_mean_pivot.loc[resorting_idx]

# Save to CSV
lightsheet_data_mean_pivot.to_csv('suppTable_mean.csv')